In [14]:
import pandas as pd
base = pd.read_csv('house_prices.csv')
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [15]:
base.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [16]:
colunas_usadas = ['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']
colunas_usadas

['price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long']

In [17]:
#Recarregando a base de dados utilizando somente as colunas indicadas

base = pd.read_csv('house_prices.csv', usecols = colunas_usadas)

In [18]:
# Aplicação da normalização nos atributos previsores

from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()

base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']] = scaler_x.fit_transform(base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']])

scaler_y = MinMaxScaler()
base[['price']] = scaler_y.fit_transform(base[['price']]) ## normalizando o Y

In [19]:
x = base.drop('price', axis=1) # atributos previsores
y = base.price # variável dependente
y

0        0.019266
1        0.060721
2        0.013770
3        0.069377
4        0.057049
           ...   
21608    0.037377
21609    0.042623
21610    0.042898
21611    0.042623
21612    0.032787
Name: price, Length: 21613, dtype: float64

In [8]:
previsores_colunas = colunas_usadas[1:17]
previsores_colunas

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long']

In [9]:
import tensorflow as tf

C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [10]:
# Feature colunm (recebendo o nome das colunas)
colunas = [tf.feature_column.numeric_column(key = c) for c in previsores_colunas] 

In [11]:
from sklearn.model_selection import train_test_split # separando base de dados, 70% treinamento e 30% teste
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x,y, test_size = 0.3)

In [12]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = x_treinamento, y = y_treinamento,
                                                        batch_size = 32, num_epochs = None, shuffle = True)

In [13]:
regressor = tf.estimator.DNNRegressor(hidden_units = [8, 8, 8], feature_columns = colunas)
regressor.train(input_fn = funcao_treinamento, steps = 20000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Douglas\\AppData\\Local\\Temp\\tmpjg1vnu29', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000254DEC47DC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Var

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Douglas\AppData\Local\Temp\tmpjg1vnu29\model.ckpt.
INFO:tensorflow:loss = 0.42487386, step = 1
INFO:tensorflow:global_step/sec: 324.528
INFO:tensorflow:loss = 0.017487975, step = 101 (0.313 sec)
INFO:tensorflow:global_step/sec: 359.837
INFO:tensorflow:loss = 0.01641864, step = 201 (0.276 sec)
INFO:tensorflow:global_step/sec: 371.72
INFO:tensorflow:loss = 0.023714427, step = 301 (0.269 sec)
INFO

INFO:tensorflow:loss = 0.008876095, step = 6101 (0.266 sec)
INFO:tensorflow:global_step/sec: 368.977
INFO:tensorflow:loss = 0.005803196, step = 6201 (0.273 sec)
INFO:tensorflow:global_step/sec: 370.344
INFO:tensorflow:loss = 0.011010607, step = 6301 (0.267 sec)
INFO:tensorflow:global_step/sec: 378.759
INFO:tensorflow:loss = 0.012180703, step = 6401 (0.265 sec)
INFO:tensorflow:global_step/sec: 386.072
INFO:tensorflow:loss = 0.012767, step = 6501 (0.258 sec)
INFO:tensorflow:global_step/sec: 377.331
INFO:tensorflow:loss = 0.035863597, step = 6601 (0.266 sec)
INFO:tensorflow:global_step/sec: 373.107
INFO:tensorflow:loss = 0.0024060435, step = 6701 (0.267 sec)
INFO:tensorflow:global_step/sec: 373.107
INFO:tensorflow:loss = 0.005740879, step = 6801 (0.268 sec)
INFO:tensorflow:global_step/sec: 378.76
INFO:tensorflow:loss = 0.008898915, step = 6901 (0.266 sec)
INFO:tensorflow:global_step/sec: 377.331
INFO:tensorflow:loss = 0.012668825, step = 7001 (0.264 sec)
INFO:tensorflow:global_step/sec: 3

INFO:tensorflow:loss = 0.019556234, step = 14201 (0.286 sec)
INFO:tensorflow:global_step/sec: 371.721
INFO:tensorflow:loss = 0.013175366, step = 14301 (0.270 sec)
INFO:tensorflow:global_step/sec: 368.976
INFO:tensorflow:loss = 0.02727636, step = 14401 (0.273 sec)
INFO:tensorflow:global_step/sec: 332.201
INFO:tensorflow:loss = 0.011764854, step = 14501 (0.299 sec)
INFO:tensorflow:global_step/sec: 345.995
INFO:tensorflow:loss = 0.011847483, step = 14601 (0.291 sec)
INFO:tensorflow:global_step/sec: 377.332
INFO:tensorflow:loss = 0.009650314, step = 14701 (0.264 sec)
INFO:tensorflow:global_step/sec: 349.623
INFO:tensorflow:loss = 0.0053686397, step = 14801 (0.285 sec)
INFO:tensorflow:global_step/sec: 336.677
INFO:tensorflow:loss = 0.025708422, step = 14901 (0.297 sec)
INFO:tensorflow:global_step/sec: 368.977
INFO:tensorflow:loss = 0.014737321, step = 15001 (0.272 sec)
INFO:tensorflow:global_step/sec: 334.424
INFO:tensorflow:loss = 0.003922482, step = 15101 (0.298 sec)
INFO:tensorflow:globa

KeyboardInterrupt: 

In [ ]:
funcao_previsao = tf.estimator.inputs.pandas_input_fn(x = x_teste, shuffle = False)
previsoes = regressor.predict(input_fn = funcao_previsao)
list(previsoes)

In [ ]:
valores_previsoes = []
for p in regressor.predict(input_fn = funcao_previsao):
    valores_previsoes.append(p['predictions'][0])

In [ ]:
valores_previsoes

In [ ]:
import numpy as np
valores_previsoes = np.asarray(valores_previsoes).reshape(-1,1)
valores_previsoes = scaler_y.inverse_transform(valores_previsoes)
valores_previsoes

In [ ]:
y_teste2 = y_teste.values.reshape(-1,1)
y_teste2 = scaler_y.inverse_transform(y_teste2)
y_teste2

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_teste2, valores_previsoes)
mae

In [ ]:
# se observa em relação a outro exemplo sem usar redes neurais que o valor do erro foi de 130k

# ganho considerável